# 🧠 Project 01 — Artificial Intelligence 2025/26  
## Class Timetable (CSP Problem)

---

### 1. Introduction
> This project aims to develop an intelligent agent capable of generating class timetables for the undergraduate courses at the Polytechnic Institute of Cávado and Ave (IPCA).  
> Creating timetables is a complex and time-consuming task that involves multiple constraints related to professors, subjects, classrooms, and student groups. Traditionally, this process requires significant time and effort from the administrative team.  
> In this context, the project intends to apply Artificial Intelligence techniques, specifically Constraint Satisfaction Problems (CSP), to automate the creation of viable and optimized timetables, respecting the imposed conditions and minimizing conflicts.  
> The project will be implemented in Python, using the `python-constraint` library, and fully documented in a Jupyter Notebook, as per the guidelines of the Artificial Intelligence (AI) course.

**Group members:**
- Pedro Ribeiro — student number 27960  
- Ricardo Fernandes — student number 27961  
- Carolina Branco — student number 27983  
- João Barbosa — student number 27964  
- Diogo Abreu — student number 27975  

---

### 2. Goal Formulation

The main goal of this project is to **design an intelligent agent** capable of automatically generating valid class timetables for undergraduate courses at IPCA.  

The agent must assign **courses to time slots and rooms**, ensuring that no scheduling conflicts occur and that all academic and logistical constraints are satisfied.  

#### Limitations and Constraints
The timetable generation process is subject to several limitations, including:

- **Teacher availability** — each lecturer may have unavailable time slots.  
- **Room capacity and restrictions** — some courses must occur in specific rooms (e.g., labs).  
- **Course assignment** — each class is associated with a specific set of courses.  
- **Online lessons** — some sessions may occur online and not require a physical room.  
- **Non-overlapping sessions** — a class, teacher, or room cannot be used in two sessions simultaneously.  
- **Class duration** — each lesson lasts 2 hours.  
- **Weekly lessons per class** — all classes have 10 lessons per week.  
- **Lessons per course** — each course may have 1 or 2 lessons per week.  
- **Daily lesson limit** — a class cannot have more than 3 lessons per day.  

#### Expected Results
The system should produce a **valid timetable solution** that:
- Assigns every course session to a valid time slot and room.  
- Respects all **hard constraints**.  
- Minimizes conflicts and overlaps between classes and teachers.  
- Optionally considers **soft constraints** to optimize the timetable.

---

### 3. Problem Formulation (CSP)

The timetable problem was modeled as a **Constraint Satisfaction Problem (CSP)** using the `python-constraint` library.

#### Variables
Each **lesson** of a course is represented by two variables:
- `<course>_<lesson>_slot` — the time slot assigned to that lesson.  
- `<course>_<lesson>_room` — the classroom assigned to that lesson.

#### Domains
- **Time slots:** `1–20` (representing possible periods in the week).  
- **Rooms:** `{Lab01, Room1, Room2}`, unless specific restrictions apply.  

#### Constraints

**Hard Constraints (mandatory):**
- Classes last 2 hours.  
- All classes have 10 lessons per week.  
- Each course may have 1 or 2 lessons per week.  
- A class cannot have more than 3 lessons per day.  
- Teacher availability — a course cannot be scheduled in time slots where its lecturer is unavailable.  
- Room restrictions — some courses are restricted to specific rooms (e.g., labs).  
- No overlapping classes — no class, teacher, or room can appear in two lessons at the same time.  

**Soft Constraints (preferred, can be violated if necessary):**
- Lessons of the same course must be scheduled on distinct days.  
- Each class should have only four days of lessons per week.  
- Lessons within the same day should be consecutive. 

#### Additional Constraints (optional improvements)

**Hard Constraints (additional):**
- When online classes are scheduled, limited to a maximum of three per course, they must be scheduled on the same day.  
- Some classes are required to be assigned to a specific classroom (already partly implemented via `roomrestrictions`).  

**Soft Constraints (additional):**
- The number of classrooms used by each class should be minimized.  

#### Heuristics
The CSP solver may apply **variable ordering heuristics** (e.g., most constrained variable first) and **domain reduction** to improve efficiency.  
Further heuristics (e.g., least constraining value) may be explored in later iterations.

---

### 5. Repository Link
> 🔗 **GitHub Repository:** [https://github.com/diogooaabreu/IA25_P01_G4.git](https://github.com/diogooaabreu/IA25_P01_G4.git)

---

### 4. Implementation

> The implementation is based on the `python-constraint` library.  
> The dataset is read from `datasets/timetable_dataset.txt`, and constraints are dynamically built from the data.  
> Below is the initial implementation of the CSP model:

---




In [ ]:
from constraint import *
from datasets.constants import *
import itertools 

#1. Import datasets

dataset_path = "datasets/timetable_dataset.txt"

courses_assigned_to_classes = {}
courses_assigned_to_lecturers = {}
timeslot_restrictions = {}
roomrestrictions = {}
online_classes = {}

with open(dataset_path, "r") as f:
    lines = f.readlines()

reading_section = None

for line in lines:
    line = line.strip()
    if not line:
        continue
    if line.startswith("#cc"):
        reading_section = "cc"
        continue
    elif line.startswith("#dsd"):
        reading_section = "dsd"
        continue
    elif line.startswith("#tr"):
        reading_section = "tr"
        continue
    elif line.startswith("#rr"):
        reading_section = "rr"
        continue
    elif line.startswith("#oc"):
        reading_section = "oc"
        continue
    elif line.startswith("#"):
        reading_section = None
        continue

    if reading_section == "cc":
        parts = line.split()
        class_name = parts[0]
        courses = parts[1:]
        courses_assigned_to_classes[class_name] = courses
    elif reading_section == "dsd":
        parts = line.split()
        teacher_name = parts[0]
        courses = parts[1:]
        courses_assigned_to_lecturers[teacher_name] = courses
    elif reading_section == "tr":
        parts = line.split()
        teacher = parts[0]
        courses = list(map(int, parts[1:]))
        timeslot_restrictions[teacher] = courses
    elif reading_section == "rr":
        parts = line.split()
        course = parts[0]
        room = parts[1:]
        roomrestrictions[course] = room
    elif reading_section == "oc":
        parts = line.split()
        course = parts[0]
        lesson_week_index = parts[1:]
        online_classes[course] = lesson_week_index


print("Class courses:", courses_assigned_to_classes)
print("Teachers courses:", courses_assigned_to_lecturers)
print("Teacher unavailable slots:", timeslot_restrictions)
print("Room restrictions:", roomrestrictions)
print("Online classes:", online_classes)

# =============================
# 2. Create CSP (Constraint Satisfaction Problem)
# =============================

# Inicializa o solver do CSP usando a biblioteca python-constraint
problem = Problem()

# Lista de salas disponíveis. Incluí "Online" para aulas que não requerem sala física.
rooms = ["Lab01", "Room1", "Room2", "Online"]

# Número de aulas por curso (assumido constante = 2)
lessons_per_course = 2

# Lista de slots horários possíveis (1 a 20, assumindo 5 dias x 4 períodos por dia)
slots = list(range(1, 21))

# ---------------------------------
# 1- Criação das variáveis do CSP
# Cada aula de cada curso tem duas variáveis:
# - slot: horário em que a aula ocorre
# - room: sala onde a aula ocorre
# ---------------------------------
for class_name, courses in courses_assigned_to_classes.items():
    for course in courses:
        for lesson in range(1, lessons_per_course + 1):
            problem.addVariable(f"{course}_{lesson}_slot", slots)
            if course in roomrestrictions:
               problem.addVariable(f"{course}_{lesson}_room", roomrestrictions[course])
            else:
                problem.addVariable(f"{course}_{lesson}_room", rooms)
                
            teachers_for_course = [t for t, c_list in courses_assigned_to_lecturers.items() if course in c_list]
            problem.addVariable(f"{course}_{lesson}_teacher", teachers_for_course)


# ---------------------------------
# 2️- Constraint: Não permitir slots consecutivos para a mesma turma
# Aqui assume-se que a função `no_consecutive_slots` está definida noutro módulo
# ---------------------------------
for class_name, courses in courses_assigned_to_classes.items():
    lesson_vars = [f"{course}_{lesson}_slot" for course in courses for lesson in range(1, lessons_per_course + 1)]
    problem.addConstraint(no_consecutive_slots, lesson_vars)


# ---------------------------------
# 3️- Constraint: Evitar sobreposição de aulas na mesma sala
# A função room_no_consecutive impede que duas aulas ocupem a mesma sala ao mesmo tempo
# Este bloco adiciona esta restrição para cada sala
# ---------------------------------
for room in rooms:
    room_slot_vars = []
    for class_name, courses in courses_assigned_to_classes.items():
        for course in courses:
            for lesson in range(1, lessons_per_course + 1):
                room_slot_vars.append((f"{course}_{lesson}_room", f"{course}_{lesson}_slot"))
    
    var_names = []
    for pair in room_slot_vars:
        var_names.extend(pair)
    
    problem.addConstraint(lambda *args, r=room: room_no_consecutive(r, *args), var_names)

# ---------------------------------
# 4️- Constraint: Disponibilidade dos professores
# Nenhuma aula pode ser atribuída a um slot em que o professor está indisponível
# ---------------------------------
for teacher, unavailable_slots in timeslot_restrictions.items():
    if teacher in courses_assigned_to_lecturers:
        for course in courses_assigned_to_lecturers[teacher]:
            for lesson in range(1, lessons_per_course + 1):
                problem.addConstraint(
                    lambda s, unav=unavailable_slots: s not in unav,
                    (f"{course}_{lesson}_slot",)
                )


# ---------------------------------
# 5️- Constraint: Evitar conflito de sala em slots específicos
# Verifica se a mesma sala não é usada em simultâneo em qualquer slot
# ---------------------------------
for room in rooms:
    for slot in slots:
        def room_slot_constraint(*args, r=room, s=slot):
            # conta quantas vezes a sala r aparece no slot s
            count = 0
            for i in range(0, len(args), 2):
                if args[i] == r and args[i+1] == s:
                    count += 1
                    if count > 1:
                        return False
            return True

        vars_for_room = []
        for class_name, courses in courses_assigned_to_classes.items():
            for course in courses:
                for lesson in range(1, lessons_per_course + 1):
                    vars_for_room.append(f"{course}_{lesson}_room")
                    vars_for_room.append(f"{course}_{lesson}_slot")
        
        problem.addConstraint(room_slot_constraint, vars_for_room)


# ---------------------------------
# 6️- Constraint: Turmas não podem ter duas aulas no mesmo slot
# Garante que aulas da mesma turma não se sobrepõem
# ---------------------------------
for class_name, courses in courses_assigned_to_classes.items():
    lesson_vars = [f"{course}_{lesson}_slot" for course in courses for lesson in range(1, lessons_per_course + 1)]
    problem.addConstraint(AllDifferentConstraint(), lesson_vars)


# ---------------------------------
# 7️- Constraint: Professores não podem dar duas aulas ao mesmo tempo
# Garante que um professor não é alocado em slots diferentes simultaneamente
# ---------------------------------
for teacher, courses in courses_assigned_to_lecturers.items():
    lesson_vars = [f"{course}_{lesson}_slot" for course in courses for lesson in range(1, lessons_per_course + 1)]
    problem.addConstraint(AllDifferentConstraint(), lesson_vars)

# ---------------------------------
# 8️- Constraint: Aulas online
# Se o curso tiver aulas online, força a sala a ser "Online"
# ---------------------------------
for course, week_indexes in online_classes.items():
    for lesson in week_indexes:
        problem.addConstraint(
            lambda r: r == "Online",
            (f"{course}_{lesson}_room",)
        )


# ---------------------------------
# 9️- Constraint: Limite máximo de 3 aulas por dia para cada turma
# Funções auxiliares:
# - get_day_from_slot: converte slot em dia (0-4)
# - max_three_lessons_per_day: verifica limite diário
# ---------------------------------
def get_day_from_slot(slot):
    return (slot - 1) // 4

def max_three_lessons_per_day(*slots):
    from collections import Counter
    days = [get_day_from_slot(s) for s in slots]
    c = Counter(days)
    return all(v <= 3 for v in c.values())

for class_name, courses in courses_assigned_to_classes.items():
    lesson_vars = [f"{course}_{lesson}_slot" for course in courses for lesson in range(1, lessons_per_course + 1)]
    problem.addConstraint(max_three_lessons_per_day, lesson_vars)

###
###print("Variáveis do CSP:")
###for var_name in problem._variables:
###    print(var_name, "→ domínio:", problem._variables[var_name])

# ---------------------------------
# 10️- Resolver o CSP
# O solver tenta encontrar uma solução que satisfaça todas as constraints
# ---------------------------------
solution = problem.getSolution()
if solution:
    print("\n✅ Solução encontrada:")
    for var in sorted(solution):
        print(var, "→", solution[var])
else:
    print("\n❌ Nenhuma solução encontrada")


Class courses: {'t01': ['UC11', 'UC12', 'UC13', 'UC14', 'UC15']}
Teachers courses: {'jo': ['UC11', 'UC21', 'UC22', 'UC31'], 'mike': ['UC12', 'UC23', 'UC32'], 'rob': ['UC13', 'UC14', 'UC24', 'UC33'], 'sue': ['UC15', 'UC25', 'UC34', 'UC35']}
Teacher unavailable slots: {'mike': [13, 14, 15, 16, 17, 18, 19, 20], 'rob': [1, 2, 3, 4], 'sue': [9, 10, 11, 12, 17, 18, 19, 20]}
Room restrictions: {'UC14': ['Lab01'], 'UC22': ['Lab01']}
Online classes: {'UC21': ['2'], 'UC31': ['2']}


KeyError: 'UC23_1_slot'